In [24]:
%run base.ipynb

# EPIC 1 - User Story DF-24
## Erstellung eines Notebooks und aufrufen der Daten per Dataland-API

Als QA-Lab-Team möchten wir ein Notebook erstellen, mit dem wir eine Anfrage an die Dataland-API senden, sodass wir anhand der Berichts-ID den Unternehmensbericht erhalten.

Beschreibung:<br>
Nutzen von Dataland Endpoint /data/nuclear-and-gas/companies/{companyId} im Nuclear & Gas Controller. Die Funktionalität wird außerhalb des Notebooks implementiert.

Akzeptanzkriterien:<br>
    - Notebook ist auf jedem Laptop ausführbar<br>
    - Anfrage an Dataland gibt Daten zurück

## 1.Schritt: Dataset mit Hilfe der Company_ID über die Dataland_API aufrufen
Mithilfe der Company_ID wird die Dataland_API aufgerufen. Nachdem Eintragen aus welcher Periode das Dataset sein soll, ist es möglich die Data_ID zu erlangen.

In [ ]:
import dataland_qa_lab.dataland.get_data as qa

data_id = qa.get_data_set_by_year(company_id="b3249e50-dede-4fc2-b348-8b8daf36aebe", year="2024")


'7b333da2-dd40-40ee-905f-77eab20cab05'

## 2.Schritt: Daten laden
Hier werden die Daten von Dataland geladen und alle 6 Werte gesondert gespeichert.

In [ ]:
from dataland_qa_lab.utils import config

conf = config.get_config()
dataland_client = conf.dataland_client

data = dataland_client.eu_taxonomy_nuclear_and_gas_api.get_company_associated_nuclear_and_gas_data(data_id=data_id)

wert1 = data.data.general.general.nuclear_energy_related_activities_section426
wert2 = data.data.general.general.nuclear_energy_related_activities_section427
wert3 = data.data.general.general.nuclear_energy_related_activities_section428
wert4 = data.data.general.general.fossil_gas_related_activities_section429
wert5 = data.data.general.general.fossil_gas_related_activities_section430
wert6 = data.data.general.general.fossil_gas_related_activities_section431
print(data)
print(wert1)
print(wert2)
print(wert3)
print(wert4)
print(wert5)
print(wert6)

company_id='b3249e50-dede-4fc2-b348-8b8daf36aebe' reporting_period='2024' data=NuclearAndGasData(general=NuclearAndGasGeneral(general=NuclearAndGasGeneralGeneral(referenced_reports={'Unternehmens_Bericht': CompanyReport(file_reference='b0ec716d4f5bc07f07608c58a6ea9d84c0b87e81915c0b5cc8d0d39209c6b438', file_name=None, publication_date=datetime.date(2024, 11, 17))}, nuclear_energy_related_activities_section426=ExtendedDataPointYesNo(value=<YesNo.YES: 'Yes'>, quality=<QualityOptions.REPORTED: 'Reported'>, comment='', data_source=ExtendedDocumentReference(page='1', tag_name=None, file_name='Unternehmens_Bericht', file_reference='b0ec716d4f5bc07f07608c58a6ea9d84c0b87e81915c0b5cc8d0d39209c6b438')), nuclear_energy_related_activities_section427=None, nuclear_energy_related_activities_section428=None, fossil_gas_related_activities_section429=None, fossil_gas_related_activities_section430=None, fossil_gas_related_activities_section431=None), taxonomy_aligned_denominator=NuclearAndGasGeneralTaxon

## 3.Schritt: Datenquelle laden
In diesem Fall wird die Dateireferenz von wert1 genutzt um das dazugehörige Dokument zu bekommen.

In [5]:
# datenQuelle = wert1.data_source.file_reference
# print(datenQuelle)

document_bytes = dataland_client.documents_api.get_document(wert1.data_source.file_reference)

## 4.Schritt: Daten aus dem Dokument extrahieren und in Text umwandeln

In [6]:
import io

import pypdf

full_document_byte_stream = io.BytesIO(document_bytes)
full_pdf = pypdf.PdfReader(full_document_byte_stream)

partial_document_byte_stream = io.BytesIO()
partial_pdf = pypdf.PdfWriter()

partial_pdf.add_page(full_pdf.get_page(int(wert1.data_source.page) - 1))  # Correct for 0 offset
partial_pdf.write(partial_document_byte_stream)
partial_document_byte_stream.seek(0)
None

In [7]:
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, ContentFormat
from azure.core.credentials import AzureKeyCredential

docintel_cred = AzureKeyCredential(conf.azure_docintel_api_key)
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=conf.azure_docintel_endpoint, credential=docintel_cred
)

poller = document_intelligence_client.begin_analyze_document(
    "prebuilt-layout",
    analyze_request=partial_document_byte_stream,
    content_type="application/octet-stream",
    output_content_format=ContentFormat.MARKDOWN,
)
result: AnalyzeResult = poller.result()

#### Das Ergebnis wird als Markdown direkt im Notebook wiedergegeben

In [8]:
from IPython.display import Markdown, display

display(Markdown(result.content))

Nein
